# importing

In [39]:
# Data handling
import pandas as pd
import numpy as np

# Model selection and evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import  make_scorer,mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import RandomizedSearchCV
import lightgbm as lgb




# read Data

In [40]:
import pandas as pd
df= pd.read_csv(r"D:\Downloads\MIA\Task3\f-1-racer-diet-planning\train.csv")
df.head()

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


# preprossessing 

In [41]:
x = df.drop(columns=['id', 'Calories'])
y = df['Calories']
x['Sex'] = np.where(x['Sex'] == 'male', 1, 0)
x.head()

,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
0,1,36,189.0,82.0,26.0,101.0,41.0
1,0,64,163.0,60.0,8.0,85.0,39.7
2,0,51,161.0,64.0,7.0,84.0,39.8
3,1,20,192.0,90.0,25.0,105.0,40.7
4,0,38,166.0,61.0,25.0,102.0,40.6


# splitting 

In [42]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42) 

## using LGB model (tree based model )
## perform random search to save time 

In [43]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

import numpy as np

# ----- Step 1: Randomized Search on Train/Val -----
param_dist = {
    # Number of leaves per tree (controls model complexity)
    'num_leaves': [31, 63, 127, 255],
    #  Maximum depth of trees (-1 = unlimited) 
    'max_depth': [-1, 5, 10, 15],
    # Step size shrinkage for boosting
    'learning_rate': [0.01, 0.05, 0.1],
    # Fraction of rows used for each tree (helps prevent overfitting).
    'subsample': [0.6, 0.8, 1.0],
    # Fraction of features used for each tree (helps prevent overfitting).
    'colsample_bytree': [0.6, 0.8, 1.0],
    # L1 regularization term.
    'reg_alpha': [0, 0.1, 0.5, 1.0],
    # L2 regularization term.
    'reg_lambda': [0, 0.1, 0.5, 1.0]
}

# Model for tuning
lgb_base = lgb.LGBMRegressor(
    objective='regression',
    # means we start with 300 trees during tuning
    n_estimators=300,   # Keep small for faster search
    # Boosting type
    boosting_type='gbdt',
    random_state=42
)

# Randomized Search
random_search = RandomizedSearchCV(
    estimator=lgb_base,
    param_distributions=param_dist,
    n_iter=20,  # Number of random combinations to try
    scoring='neg_root_mean_squared_error',
    cv=[(np.arange(len(x_train)), np.arange(len(x_val)) + len(x_train))],
    # parameter controls how much information is printed to the console while the model or search is running.
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# Combining the training and validation sets into a single dataset
X_train_val = np.concatenate([x_train, x_val])
y_train_val = np.concatenate([y_train, y_val])

random_search.fit(X_train_val, y_train_val)

print("Best Parameters:", random_search.best_params_)
print("Best RMSE (Val):", -random_search.best_score_)


best_params = random_search.best_params_

final_model = lgb.LGBMRegressor(
    **best_params,
    n_estimators=1000,  # More trees for final model
    random_state=42
)

final_model.fit(X_train_val, y_train_val)


y_test_pred = final_model.predict(x_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
print(f"Final Test RMSE: {test_rmse:.4f}")


Fitting 1 folds for each of 20 candidates, totalling 20 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010402 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 362
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 7
[LightGBM] [Info] Start training from score 88.298465
Best Parameters: {'subsample': 0.8, 'reg_lambda': 0.1, 'reg_alpha': 0.5, 'num_leaves': 127, 'max_depth': 15, 'learning_rate': 0.05, 'colsample_bytree': 0.8}
Best RMSE (Val): 3.5743299492863336
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003645 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 362
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 7
[LightGBM] [Info] Start training from score 88.29846

In [44]:
df_test= pd.read_csv(r"D:\Downloads\MIA\Task3\f-1-racer-diet-planning\test.csv")

x_test = df_test.drop(columns=['id'])
x_test['Sex'] = np.where(x_test['Sex'] == 'male', 1, 0)



In [45]:
y_test_pred = final_model.predict(x_test)

submission = pd.DataFrame({
    "id": df_test["id"],
    "Calories": np.round(y_test_pred, 3)
})
submission.to_csv("submissionTsk3.2lgb.csv", index=False)